In [ ]:
# !pip install autofeat

In [ ]:
# !pip install sweetviz

In [ ]:
!pip install optuna

In [ ]:
import pandas as pd

import numpy as np

from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, PolynomialFeatures, RobustScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor, VotingRegressor, HistGradientBoostingRegressor
from sklearn.linear_model import BayesianRidge, LinearRegression, LassoLars, Lars, Ridge, ElasticNet

# import sweetviz as sv

import matplotlib.pyplot as plt

import seaborn as sns

from xgboost import XGBRegressor

from catboost import CatBoostRegressor

from lightgbm import LGBMRegressor

import tqdm

from category_encoders import TargetEncoder

import optuna

# from autofeat import AutoFeatRegressor

# from pystacknet.pystacknet import StackNetRegressor

# from GML.Ghalat_Machine_Learning import Ghalat_Machine_Learning

In [ ]:
def accuracy(y_true,y_pred):
  return 100 * max(0,1-np.sqrt(mean_squared_error(y_true,y_pred)))

In [ ]:
path = '/kaggle/input/he-mlcomp/'

In [ ]:
train = pd.read_csv(path+'Train.csv')
test = pd.read_csv(path+'Test.csv')

In [ ]:
"""
test = train[5000:].copy()
test_labels = test['Attrition_rate']
test.drop(['Attrition_rate'],axis=1,inplace=True)
train = train[:5000].copy()
"""
pass

In [ ]:
train.columns

In [ ]:
train.head()

In [ ]:
test_eid = test['Employee_ID'].copy()

In [ ]:
train['Salary_Now'] = train['Pay_Scale'] * 10000
train['Salary_In_Next_Year'] = train['Salary_Now'] + train['Salary_Now'] * (train['growth_rate']/100)
train['Salary_In_Next_10_Years'] = train['Salary_Now'] + train['Salary_Now'] * (train['growth_rate']/100)*10

test['Salary_Now'] = test['Pay_Scale'] * 10000
test['Salary_In_Next_Year'] = test['Salary_Now'] + test['Salary_Now'] * (test['growth_rate']/100)
test['Salary_In_Next_10_Years'] = test['Salary_Now'] + test['Salary_Now'] * (test['growth_rate']/100)*10

In [ ]:
def UxH(x):
    return str(x['Unit']) + '_' + str(x['Hometown'])

def XxBxH(x):
    return str(x['Compensation_and_Benefits']) + '_' + str(x['Hometown'])

def XxBxU(x):
    return str(x['Compensation_and_Benefits']) + '_' + str(x['Unit'])

def emp_Type(s):
    if s['Time_of_service'] == 0.0:
        return "New"
    elif s['Time_of_service'] <= 5.0:
        return "Moderate"
    elif s['Time_of_service'] <= 10.0:
        return "Senior"
    else:
        return "SuperSenior"
    
def person_Type(s):
    if s['Age'] <= 20:
        return 'Teenager'
    elif s['Age'] <= 40:
        return "Man"
    elif s['Age'] <= 60:
        return "Uncle"
    else:
        return "AboutToDie"

In [ ]:
train['Emp_Type'] = train.apply(emp_Type,axis=1)

train['person_Type'] = train.apply(person_Type,axis=1)

train['UxH'] = train.apply(UxH,axis=1)

test['Emp_Type'] = test.apply(emp_Type,axis=1)

test['person_Type'] = test.apply(person_Type,axis=1)

test['UxH'] = test.apply(UxH,axis=1)

In [ ]:
pvar = 'Hometown'

In [ ]:
train['Perc_Spent_WO_Promotion'] = train['Time_since_promotion'] / train['Time_of_service']
test['Perc_Spent_WO_Promotion'] = test['Time_since_promotion'] / test['Time_of_service']

In [ ]:
le = LabelEncoder()
train[pvar] = le.fit_transform(train[pvar])
test[pvar] = le.fit_transform(test[pvar])

In [ ]:
train['Employee_ID'] = train['Employee_ID'].apply(lambda x: int(x.split('_')[1]))
test['Employee_ID'] = test['Employee_ID'].apply(lambda x: int(x.split('_')[1]))

In [ ]:
def age_bin(s):
    return s['Age']/10

def emp_bin(s):
    if s['Employee_ID'] <= 7200:
        return 'OLD'
    if s['Employee_ID'] <= 18000:
        return 'MID'
    else:
        return 'NEW'

def growth_bin(s):
    if s['growth_rate'] <= 33:
        return 'NOT Satisfy'
    if s['growth_rate'] <= 47:
        return 'Satisfy'
    else:
        return 'High Satisfy'

train['age_bins'] = train.apply(age_bin,axis=1)

train['emp_bins'] = train.apply(emp_bin,axis=1)

train['growth_bins'] = train.apply(growth_bin,axis=1)

test['age_bins'] = test.apply(age_bin,axis=1)

test['emp_bins'] = test.apply(emp_bin,axis=1)

test['growth_bins'] = test.apply(growth_bin,axis=1)

In [ ]:
types = {
    'type0':'M2',
    'type1':'M2',
    'type2':'L2',
    'type4':'L2',
    'type3':'L2'
}

train['COMPType'] = train['Compensation_and_Benefits'].map(types)
test['COMPType'] = test['Compensation_and_Benefits'].map(types)

In [ ]:
cat_features = list(train.columns[train.dtypes == 'object'])

In [ ]:
for i in range(1,8):
    name = 'VAR'+str(i)
    cat_features.append(name)

In [ ]:
for cat in cat_features:
    g_var = cat
    mediann = dict(train.groupby(g_var)['Attrition_rate'].mean())
    train[str(cat)+'_Avg'] = train[g_var].map(mediann)
    test[str(cat)+'_Avg'] = test[g_var].map(mediann)

In [ ]:
cols = train.columns[train.dtypes == 'object']
for col in cols:
    train[col].interpolate(method='linear',inplace=True)
    test[col].interpolate(method='linear',inplace=True)

In [ ]:
test.sort_values('Age',ascending=False,inplace=True)
train.sort_values('Age',ascending=False,inplace=True)

In [ ]:
def age_bin(s):
    return s['Age']/10

def emp_bin(s):
    if s['Employee_ID'] <= 7200:
        return 'OLD'
    if s['Employee_ID'] <= 18000:
        return 'MID'
    else:
        return 'NEW'

def growth_bin(s):
    if s['growth_rate'] <= 33:
        return 'NOT Satisfy'
    if s['growth_rate'] <= 47:
        return 'Satisfy'
    else:
        return 'High Satisfy'


In [ ]:
train['age_bins'] = train.apply(age_bin,axis=1)
test['age_bins'] = test.apply(age_bin,axis=1)

train['emp_bins'] = train.apply(emp_bin,axis=1)
test['emp_bins'] = test.apply(emp_bin,axis=1)

train['growth_bins'] = train.apply(growth_bin,axis=1)
test['growth_bins'] = test.apply(growth_bin,axis=1)

In [ ]:
grid_lgbm = {
    'max_depth':16,
    'learning_rate':0.002,
    'n_estimators':1704,
    'min_child_weight':17,
    'eta':0.364
}

In [ ]:
cols = train.columns[train.dtypes == 'object']
encoded = pd.DataFrame([])

X = train[cols]
y = train['Attrition_rate'].copy().values

for tr_in, val_in in KFold(shuffle = True).split(X, y):
    enc = TargetEncoder()
    enc.fit(X.iloc[tr_in], y[tr_in])
    encoded = encoded.append(enc.transform(X.iloc[val_in]))

enc = TargetEncoder()
train[cols] = enc.fit(X[cols],y)
test[cols] = enc.transform(test[cols])
train[cols] = encoded.sort_index()
"""
for col in cols:
  dummies = pd.get_dummies(train[col])
  train.drop([col],axis=1,inplace=True)
  train = pd.concat([train,dummies],axis=1)
  dummies = pd.get_dummies(test[col])
  test.drop([col],axis=1,inplace=True)
  test = pd.concat([test,dummies],axis=1)
"""

"""
for col in test.columns:
    if train[col].isnull().any():
        model = LGBMRegressor(**grid_lgbm)
        t1 = train[~pd.isna(train[col])].copy()
        t1.drop(['Attrition_rate'],axis=1,inplace=True)
        t2 = train[pd.isna(train[col])].copy()
        t2.drop(['Attrition_rate','Age'],axis=1,inplace=True)
        t3 = test[pd.isna(test[col])].copy()
        t3.drop(['Age'],axis=1,inplace=True)
        model.fit(t1.drop([col],axis=1),t1[col])
        train.loc[t2.index,col] = model.predict(t2)
        test.loc[t3.index,col] = model.predict(t3)
"""
"""
for cols in test.columns:
    train[cols].fillna(train[cols].mean(),inplace=True)
    test[cols].fillna(test[cols].mean(),inplace=True)
"""
pass

In [ ]:
funcs = [np.multiply, np.add, np.subtract, np.divide]

for i, func in enumerate(funcs):
    train['Age_TOS'+str(i+1)] = func(train['Age'],train['Time_of_service'])
    test['Age_TOS'+str(i+1)] = func(test['Age'],test['Time_of_service'])

    train['AT_GR'+str(i+1)] = func(train['Age_TOS'+str(i+1)], train['growth_rate'])
    test['AT_GR'+str(i+1)] = func(test['Age_TOS'+str(i+1)], test['growth_rate'])

    train['TS_GR'+str(i+1)] = func(train['Time_of_service'], train['growth_rate'])
    test['TS_GR'+str(i+1)] = func(test['Time_of_service'], test['growth_rate'])

    train['PS_WLB'+str(i+1)] = func(train['Pay_Scale'], train['Work_Life_balance'])
    test['PS_WLB'+str(i+1)] = func(test['Pay_Scale'], test['Work_Life_balance'])

    train['Age_TSP'+str(i+1)] = func(train['Age'], train['Time_since_promotion'])
    test['Age_TSP'+str(i+1)] = func(test['Age'],test['Time_since_promotion'])

    train['Age_EMP'+str(i+1)] = func(train['Age'], train['Employee_ID'])
    test['Age_EMP'+str(i+1)] = func(test['Age'], test['Employee_ID'])

    train['TOS_EMP'+str(i+1)] = func(train['Time_of_service'],train['Employee_ID'])
    test['TOS_EMP'+str(i+1)] = func(test['Time_of_service'],test['Employee_ID'])

    train['ATOS_EMP'+str(i+1)] = func(train['Age_TOS'+str(i+1)], train['Employee_ID'])
    test['ATOS_EMP'+str(i+1)] = func(test['Age_TOS'+str(i+1)], test['Employee_ID'])


var_names = ['VAR'+str(i) for i in range(1,8)]
sums_tr = []
sums_ts = []
for row in train[var_names].itertuples():
    sums_tr.append(row[1]+row[2]+row[3]+row[4]+row[5]+row[6]+row[7])
    
for row2 in test[var_names].itertuples():
    sums_ts.append(row2[1]+row2[2]+row2[3]+row2[4]+row2[5]+row2[6]+row2[7])
    
train['sum_VARS'] = sums_tr
test['sum_VARS'] = sums_ts

for col in test.columns:
    try:
        train[col].replace([np.inf,-np.inf],np.nan,inplace=True)
        test[col].replace([np.inf,-np.inf],np.nan,inplace=True)
        train[col].interpolate(method='linear',inplace=True)
        test[col].interpolate(method='linear',inplace=True)
    except:
        continue

In [ ]:
X = train.drop(['Attrition_rate'],axis=1)
y = train['Attrition_rate'].copy().values

In [ ]:
grid_bayesian = {
    'n_iter':10000,
    'tol':6.42e-06,
    'alpha_1':0.0001,
    'alpha_2':6.58e-05,
    'lambda_1':0.999,
    'lambda_2':2.24e-05,
    'compute_score':True,
    'fit_intercept':True,
    'normalize':False
}

grid_ridge = {
    'max_iter':90000,
    'alpha': 0.23143352151759083, 
    'fit_intercept': True, 
    'normalize': False, 
    'tol': 0.0005243805101392493, 
    'solver': 'saga'
}

In [ ]:
X = X.replace([np.inf, -np.inf], np.nan)
test = test.replace([np.inf, -np.inf], np.nan)
for cols in X.columns:
    X[cols].fillna(X[cols].mean(),inplace=True)
    test[cols].fillna(X[cols].mean(),inplace=True)

In [ ]:
lgbm = LGBMRegressor(**grid_lgbm)
lgbm.fit(X, y)

fi = pd.DataFrame()
fi['A'] = X.columns.values
fi['B'] = lgbm.feature_importances_

In [ ]:
fi.sort_values('B',ascending = False)

In [ ]:
fi.describe()

In [ ]:
cols_to_d = fi[fi['B'] < 10].loc[:,'A']
X.drop(cols_to_d,axis=1,inplace=True)
test.drop(cols_to_d,axis=1,inplace=True)

In [ ]:
def perc(trial):
    p1 = trial.suggest_uniform('Bayesian',0.1,0.99)
    p2 = trial.suggest_uniform('Ridge',0.1,0.99)
    p3 = trial.suggest_uniform('DecisionTree',0.1,0.99)
    p4 = trial.suggest_uniform('Lasso',0.1,0.99)
    
    _scores = []
    for v in tqdm.tqdm(np.unique(train[pvar])):
        scores = []
        X_ = X[X[pvar]==v].copy()
        indexes = X[X[pvar]==v].index
        y_ = y[indexes]
        
        for tr_in, val_in in KFold().split(X_, y_):
            ridge = Ridge(**grid_ridge)
            br = BayesianRidge()
            dt = DecisionTreeRegressor()
            lasso = LassoLars()
            
            br.fit(X_.iloc[tr_in], y_[tr_in])
            ridge.fit(X_.iloc[tr_in], y_[tr_in])
            dt.fit(X_.iloc[tr_in], y_[tr_in])
            lasso.fit(X_.iloc[tr_in], y_[tr_in])
            
            preds = (br.predict(X_.iloc[val_in]) * p1 + ridge.predict(X_.iloc[val_in]) * p2 +
                    dt.predict(X_.iloc[val_in]) * p3 + lasso.predict(X_.iloc[val_in]) * p4 )
            scores.append(accuracy(y_[val_in],preds))
        _scores.append(np.mean(scores))
    return -np.mean(_scores)

In [ ]:
# study = optuna.create_study()

In [ ]:
# study.optimize(perc)

In [ ]:
# study.best_params

In [ ]:
p1 = 0.1
p2 = 0.0
p3 = 0.0
p4 = 0.9

In [ ]:
ss = pd.DataFrame()
ss['Emp_ID'] = test['Employee_ID'].copy()
ss['Employee_ID'] = test_eid
ss['Attrition_rate'] = np.nan

In [ ]:
grid_lasso = {
     'max_iter':10000,
     'alpha': 0.0009336243734141289,
     'fit_intercept': True,
     'normalize': True,
     'eps': 5.585561275248686e-05,
     'fit_path': True,
     'positive': False,
     'jitter': 3.115210172247423e-06
}

In [ ]:
_scores = []
for v in tqdm.tqdm(np.unique(train[pvar])):
    scores = []
    X_ = X[X[pvar]==v].copy()
    X_.drop(pvar,axis=1,inplace=True)
    indexes = X[X[pvar]==v].index
    y_ = y[indexes]
    try:
        for tr_in, val_in in KFold().split(X_, y_):
            ridge = Ridge(**grid_ridge)
            br = BayesianRidge()
            dt = DecisionTreeRegressor()
            lasso = LassoLars()
            
            br.fit(X_.iloc[tr_in], y_[tr_in])
            ridge.fit(X_.iloc[tr_in], y_[tr_in])
            dt.fit(X_.iloc[tr_in], y_[tr_in])
            lasso.fit(X_.iloc[tr_in], y_[tr_in])
            
            preds = (br.predict(X_.iloc[val_in]) * p1 + ridge.predict(X_.iloc[val_in]) * p2 +
                    dt.predict(X_.iloc[val_in]) * p3 + lasso.predict(X_.iloc[val_in]) * p4 )
            scores.append(accuracy(y_[val_in],preds))
        _scores.append(np.mean(scores))
    except:
        pass
    ridge = Ridge(**grid_ridge)
    br = BayesianRidge()
    dt = DecisionTreeRegressor()
    lasso = LassoLars()

    br.fit(X_, y_)
    ridge.fit(X_, y_)
    dt.fit(X_, y_)
    lasso.fit(X_, y_)
    
    test_ = test[test[pvar]==v].copy()
    if test_.empty:
        continue
    test_.drop(pvar,axis=1,inplace=True)
    e_ind = test[test[pvar]==v].loc[:,'Employee_ID']
    
    preds = (br.predict(test_) * p1 + ridge.predict(test_) * p2 + dt.predict(test_) * p3 + 
             lasso.predict(test_) * p4)
    
    ind = ss[ss['Emp_ID'].isin(e_ind)].index
    ss.at[ind,'Attrition_rate'] = preds

In [ ]:
"""
def opti(trial):
    
    grid_lasso = {
        'alpha' : trial.suggest_loguniform('alpha',1e-6,1),
        'fit_intercept': trial.suggest_categorical('fit_intercept',[True,False]),
        'normalize':trial.suggest_categorical('normalize',[True,False]),
        'max_iter':1000000,
        'eps':trial.suggest_loguniform('eps',1e-6,1),
        'fit_path':trial.suggest_categorical('fit_path',[True,False]),
        'positive':trial.suggest_categorical('positive',[True,False]),
        'jitter':trial.suggest_loguniform('jitter',1e-6,1)
    }
    
    _scores = []
    for v in tqdm.tqdm(np.unique(train[pvar])):
        scores = []
        X_ = X[X[pvar]==v].copy()
        X_.drop(pvar,axis=1,inplace=True)
        indexes = X[X[pvar]==v].index
        y_ = y[indexes]
        try:
            for tr_in, val_in in KFold().split(X_, y_):
                lasso = LassoLars(**grid_lasso)
                lasso.fit(X_.iloc[tr_in], y_[tr_in])
                preds = lasso.predict(X_.iloc[val_in])
                scores.append(accuracy(y_[val_in],preds))
            _scores.append(np.mean(scores))
        except:
            pass
    return -np.mean(_scores)


study = optuna.create_study()
study.optimize(opti)
"""
pass

In [ ]:
_scores

In [ ]:
np.mean(_scores)

In [ ]:
ss.drop(['Emp_ID'],axis=1,inplace=True)

In [ ]:
ss = ss.sort_index()

In [ ]:
ss.to_csv('submission.csv',index=False)

In [ ]:
ss

In [ ]:
# accuracy(test_labels, ss['Attrition_rate'])